
## 这是对1.b任务的预处理和
#1.a 只有有label和时间处理

1. load——data
2. 统计特征数量（清洗）、选择、分析

In [1]:
import pandas as pd
from tqdm import *
import os
import math
from datetime import timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## load_data

In [2]:
import pickle
def save_pkl(path,obj):
    with open(path, 'wb') as f:
        pickle.dump(obj,f)
        
def load_pkl(path): 
    with open(path, 'rb') as f:
        return pickle.load(f) 
x1 = load_pkl(f'x_scale.pkl')
x1 = x1[:,0,:]
x2 = load_pkl(f'lstm_embedding.pkl').reshape(160, -1)
x3 = load_pkl(f'y.pkl')
all_data = pd.DataFrame(x1)
all_data = pd.concat( [all_data,pd.DataFrame(x2) , pd.DataFrame(x3)],axis=1)
all_data.columns = list(range(all_data.shape[1]))
all_data= all_data.rename(columns={52:'label'})

In [3]:
columns_dict={}
columns_name= ['年龄', '性别', '脑出血前mRS评分', '高血压病史', '卒中病史',
       '糖尿病史', '房颤史', '冠心病史', '吸烟史', '饮酒史', '发病到首次影像检查时间间隔', '脑室引流', '止血治疗',
       '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经', '收缩压', '舒张压'] +  ['lstm_'+str(i) for i in range(32)]
for i,value in enumerate (columns_name):
    columns_dict[i] = value


{0: '年龄',
 1: '性别',
 2: '脑出血前mRS评分',
 3: '高血压病史',
 4: '卒中病史',
 5: '糖尿病史',
 6: '房颤史',
 7: '冠心病史',
 8: '吸烟史',
 9: '饮酒史',
 10: '发病到首次影像检查时间间隔',
 11: '脑室引流',
 12: '止血治疗',
 13: '降颅压治疗',
 14: '降压治疗',
 15: '镇静、镇痛治疗',
 16: '止吐护胃',
 17: '营养神经',
 18: '收缩压',
 19: '舒张压',
 20: 'lstm_0',
 21: 'lstm_1',
 22: 'lstm_2',
 23: 'lstm_3',
 24: 'lstm_4',
 25: 'lstm_5',
 26: 'lstm_6',
 27: 'lstm_7',
 28: 'lstm_8',
 29: 'lstm_9',
 30: 'lstm_10',
 31: 'lstm_11',
 32: 'lstm_12',
 33: 'lstm_13',
 34: 'lstm_14',
 35: 'lstm_15',
 36: 'lstm_16',
 37: 'lstm_17',
 38: 'lstm_18',
 39: 'lstm_19',
 40: 'lstm_20',
 41: 'lstm_21',
 42: 'lstm_22',
 43: 'lstm_23',
 44: 'lstm_24',
 45: 'lstm_25',
 46: 'lstm_26',
 47: 'lstm_27',
 48: 'lstm_28',
 49: 'lstm_29',
 50: 'lstm_30',
 51: 'lstm_31'}

In [3]:
x1.shape

(160, 20)

In [4]:
target_column = ['label']
features =list( all_data.columns )[:-1]

# 特征选择


## 灰色关联度分析


In [5]:
train_df = all_data.head(100)
select_columns = features

In [9]:
# 计算灰度关联系数
def grey_relation_coefficient(x, y):
    x,y = np.array(x),np.array(y).ravel()
    min_x = x.min()
    max_x = x.max()
    min_y = y.min()
    max_y = y.max()
    rho = 0.15  # 灰度关联系数计算时的参数，可根据需要调整
    return np.abs((min_x - x) / (max_x - min_x) - (min_y - y) / (max_y - min_y)) / (rho + np.abs((min_x - x) / (max_x - min_x) + (min_y - y) / (max_y - min_y)))

grey_relational_matrix = pd.DataFrame()
y = (train_df[target_column].values.tolist())
for feature in select_columns:
    grey_relational_matrix[feature] = grey_relation_coefficient( list(train_df[feature]) ,list( y)) 

In [16]:

# 求平均灰度关联系数，用于特征排序
average_grey_relation = grey_relational_matrix.mean(axis=0)
average_grey_relation = average_grey_relation.sort_values(ascending=False)
average_grey_relation.describe()
# 根据灰度关联系数排序特征
ranked_features = average_grey_relation[average_grey_relation>.33].index
len(ranked_features)

30

In [17]:
select_columns = ranked_features
len(select_columns)

30

In [18]:
select_columns[:3]

Int64Index([9, 2, 8], dtype='int64')


## 弹性网回归

In [19]:
from sklearn.linear_model import ElasticNetCV

X_train = (train_df[select_columns].values)
y_train = (train_df[target_column].values)

# for i in np.linspace(0.01, 1, num=99) :
#     elastic_net = ElasticNetCV(l1_ratio=[i])  # 尝试不同的l1_ratio值
#     elastic_net.fit(X_train, y_train)
#     optimal_l1_ratio = elastic_net.l1_ratio_
#     print( i,len(np.where(elastic_net.coef_ != 0)[0]) )

elastic_net = ElasticNetCV(l1_ratio=[.2])  # 尝试不同的l1_ratio值
elastic_net.fit(X_train, y_train)
optimal_l1_ratio = elastic_net.l1_ratio_
print( .1,len(np.where(elastic_net.coef_ != 0)[0]) )


0.1 21


In [20]:
t = pd.DataFrame()
t['name'] = select_columns
t['value'] = elastic_net.coef_
t['value'] = np.abs(t['value'])
t.sort_values(['value'])

,name,value
14,15,0.000000
27,20,0.000000
25,40,0.000000
21,30,0.000000
10,12,0.000000
8,1,0.000000
29,31,0.000000
1,2,0.000000
3,6,0.000000
15,17,0.030202


In [21]:
select_columns = train_df[select_columns].iloc[:,(np.where(elastic_net.coef_ != 0)[0])].columns

## svm递归特征消除

In [22]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# 创建SVM分类器
svm = SVC(kernel="linear")


X_train = (train_df[select_columns].values)
y_train = (train_df[target_column].values)

# 使用递归特征消除来选择特征
rfe = RFE(estimator=svm, n_features_to_select=10)
rfe.fit(X_train, y_train)

# 获取选择的特征的索引
selected_features_indices = np.where(rfe.support_)[0]
selected_features = train_df[select_columns].iloc[:, selected_features_indices]
select_columns = list(selected_features.columns)
select_columns

[7, 10, 48, 39, 22, 23, 47, 37, 25, 33]

In [4]:
[columns_dict[i] for i in [7, 10, 48, 39, 22, 23, 47, 37, 25, 33] ]

['冠心病史',
 '发病到首次影像检查时间间隔',
 'lstm_28',
 'lstm_19',
 'lstm_2',
 'lstm_3',
 'lstm_27',
 'lstm_17',
 'lstm_5',
 'lstm_13']

In [23]:
all_data[select_columns+target_column ].to_csv('./data4.csv',index=False)

# 独立性检验

In [24]:
t2 = all_data[select_columns ]

In [25]:
# import scipy.stats as stats
# import itertools
# result=[]
# hyperparameter_combinations = list(itertools.product(t1.columns, t1.columns))
# for i,j in hyperparameter_combinations:
#     if(i==j):
#         continue
#     group1, group2 = t1[i],t1[j]
#     # 执行卡方独立性检验
#     contingency_table  = pd.crosstab(t1[i],t1[j])
#     chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
#     corr_coefficient, p_value1 = stats.pearsonr(group1, group2)
#     result .append([i,j,  p,p_value1   ])
# result = pd.DataFrame(result)
# for i in range(2,4):
#     print(result[result[i]<.01].shape[0])

In [26]:
import scipy.stats as stats
import itertools
result=[]
hyperparameter_combinations = list(itertools.product(t2.columns, t2.columns))
for i,j in hyperparameter_combinations:
    if(i==j):
        continue
    group1, group2 = t2[i],t2[j]
    # 执行卡方独立性检验
    contingency_table  = pd.crosstab(t2[i],t2[j])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    corr_coefficient, p_value1 = stats.pearsonr(group1, group2)
    result .append([i,j,  p,p_value1 ])
result = pd.DataFrame(result)


In [27]:
for i in range(2,4):
    print(result[result[i]<.01].shape[0])

72
56


In [28]:
result[3].describe()

count    9.000000e+01
mean     2.189921e-01
std      3.236225e-01
min      3.738609e-71
25%      2.149453e-45
50%      5.886479e-29
75%      3.104734e-01
max      9.866047e-01
Name: 3, dtype: float64

In [ ]:
# ##特征太多了 （）

# 1. 没有信息的值： 删除方差为0的(没有),使用低方差滤波器处理

# - person相关系数、speraman、kendall 来说明

# 2. 直接指标：
# -个人
# -疾病史
# -发病
# -治疗

# 方法：
# - 距离相关系数
# - 灰度关联

# 3. 带target的
# - 带LASSO 回归判断
# - svm递归特征消除

# ？ 降到多少维度

# 4. 检验
# - 独立性检验
# - 信息熵理论验证
# - 人类知识:专家知识

# 